## preparing the data for the model

In [12]:
# importing libraries and creating input data
import numpy as np
docs=['go india',
     'india india',
      'hip hip hurray',
      'jeetega bhai jeetega india jeetega',
      'bharat mata ki jai',
      'kohli kohli',
      'sachin sachin',
      'dhoni dhoni',
      'modi ji ki jai',
      'inquilab zindabad']

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token='<nothing>') #oov_nothing is to replace a word with nothing which does not exist in our train data

In [14]:
tokenizer.fit_on_texts(docs)

In [15]:
# to check what index has been alotted to the word by tokenizer
tokenizer.word_index

{'<nothing>': 1,
 'india': 2,
 'jeetega': 3,
 'hip': 4,
 'ki': 5,
 'jai': 6,
 'kohli': 7,
 'sachin': 8,
 'dhoni': 9,
 'go': 10,
 'hurray': 11,
 'bhai': 12,
 'bharat': 13,
 'mata': 14,
 'modi': 15,
 'ji': 16,
 'inquilab': 17,
 'zindabad': 18}

In [16]:
# to count the frequency of words
tokenizer.word_counts

OrderedDict([('go', 1),
             ('india', 4),
             ('hip', 2),
             ('hurray', 1),
             ('jeetega', 3),
             ('bhai', 1),
             ('bharat', 1),
             ('mata', 1),
             ('ki', 2),
             ('jai', 2),
             ('kohli', 2),
             ('sachin', 2),
             ('dhoni', 2),
             ('modi', 1),
             ('ji', 1),
             ('inquilab', 1),
             ('zindabad', 1)])

In [17]:
# to check the number of rows in document
tokenizer.document_count

10

In [18]:
# now using these tokens to generate sequences basically sentences convers to numeric form
sequences = tokenizer.texts_to_sequences(docs)
sequences

[[10, 2],
 [2, 2],
 [4, 4, 11],
 [3, 12, 3, 2, 3],
 [13, 14, 5, 6],
 [7, 7],
 [8, 8],
 [9, 9],
 [15, 16, 5, 6],
 [17, 18]]

In [19]:
# now giving padding to sentences to make them if same size
from tensorflow.keras.utils import pad_sequences
sequences = pad_sequences(sequences,padding='post') # post adds padding to the end of sentence

## Building the model to analyse sentiment

In [20]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding, Flatten

In [29]:
from sklearn.model_selection import train_test_split
data = np.load('imdb.npz',allow_pickle=True)  # Replace with your actual file path
print("Keys in the .npz file:", data.files)


Keys in the .npz file: ['x_test', 'x_train', 'y_train', 'y_test']


In [30]:
X_train = data['x_train']
X_test = data['x_test']
y_train = data['y_train']
y_test = data['y_test']

In [32]:
X_train.shape

(25000,)

In [33]:
# now lets check the length of a review
len(X_train[0])

138

In [34]:
len(X_train[1])

433

In [35]:
# as we can see the reviews are in different lengths means we need to do padding
# as we dont have enough time so we gonna take length of only 50 per review
X_train =pad_sequences(X_train,padding='post',maxlen=50)
X_test =pad_sequences(X_test,padding='post',maxlen=50)


In [36]:
X_train.shape

(25000, 50)

## Building the model

In [56]:
model = Sequential()
model.add(Embedding(10000,2))
model.add(SimpleRNN(32,input_shape=(50,1),return_sequences=False))
model.add(Dense(1,activation ='sigmoid'))

model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [38]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.5017 - loss: 0.8185 - val_accuracy: 0.5069 - val_loss: 0.6935
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.5082 - loss: 0.6929 - val_accuracy: 0.5030 - val_loss: 0.6949
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.5046 - loss: 0.6933 - val_accuracy: 0.5048 - val_loss: 0.6944
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.5072 - loss: 0.6927 - val_accuracy: 0.5064 - val_loss: 0.6934
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.5088 - loss: 0.6930 - val_accuracy: 0.5088 - val_loss: 0.6938


## using embeddings

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding

model2 = Sequential()
model2.add(Embedding(input_dim=17, output_dim=2, input_length=5))

In [52]:
model2.summary()


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [53]:
model2.compile('adam','accuracy')

In [54]:
pred=model2.predict(sequences)
print(pred)

InvalidArgumentError: Graph execution error:

Detected at node sequential_8_1/embedding_7_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 645, in run_forever

  File "C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 1999, in _run_once

  File "C